In [121]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/geotesouro/data/data.db')
cur = conn.cursor()

## Municípios - Infos

In [ ]:
import pandas as pd 

mun_git1 = pd.read_html("https://github.com/kelvins/Municipios-Brasileiros/blob/main/csv/municipios.csv")[0].drop(["Unnamed: 0"], 1)
mun_git2 = pd.read_csv("https://raw.githubusercontent.com/mapaslivres/municipios-br/main/tabelas/municipios.csv")
pop_mun = pd.read_csv("/content/drive/MyDrive/geotesouro/data/dados_municipios/pop_municipios.csv")
pib_mun = pd.read_csv("/content/drive/MyDrive/geotesouro/data/dados_municipios/pib_municipios.csv")

agg_municipios = mun_git1
merge = mun_git2[["municipio", "uf_code", "mesoregion", "microregion", "rgint", "rgi"]].rename(columns = {"municipio": "codigo_ibge", "uf_code": "uf_sigla"})
agg_municipios = agg_municipios.merge(merge, how='left', on='codigo_ibge')

pop_mun = pop_mun[["ano", "id_municipio", "populacao"]].rename(columns = {"id_municipio": "codigo_ibge"})
for a in pop_mun["ano"].unique():
  merge = pop_mun[pop_mun["ano"] == a].drop("ano", 1).rename(columns =  {"populacao": f"pop_{a}"})
  agg_municipios = agg_municipios.merge(merge, how='left', on='codigo_ibge')


pib_mun = pib_mun.rename(columns = {"id_municipio": "codigo_ibge"})
for a in pib_mun["ano"].unique():
  merge = pib_mun[pib_mun["ano"] == a].drop("ano", 1).rename(columns =  {"pib": f"pib_{a}",
                                                                         "impostos_liquidos": f"imp_liq_{a}",
                                                                         "va": f"va_{a}",
                                                                         "va_agropecuaria": f"va_agro_{a}",
                                                                         "va_industria": f"va_industria_{a}",
                                                                         "va_servicos": f"va_servicos_{a}",
                                                                         "va_adespss": f"va_adespss_{a}"})
  agg_municipios = agg_municipios.merge(merge, how='left', on='codigo_ibge')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
agg_municipios.to_sql('agg_municipios', con = conn, if_exists='replace', index=False)

## Convênios 

In [ ]:
import pandas as pd 

convenios = pd.read_csv("/content/drive/MyDrive/geotesouro/data/convenios/20220729_Convenios.csv", encoding='latin-1', sep = ";", error_bad_lines=False)

In [ ]:
convenios.to_sql('convenios', con = conn, if_exists='replace', index=False)

## Transferências 

In [ ]:
import zipfile

path_to_zip_file = "/content/drive/MyDrive/geotesouro/data/transferencias/transferencias_agg.zip"
directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais/"

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

path_to_zip_file = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais/201508_Transferencias.zip"
directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais/data/"

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import zipfile
from os import listdir
from os.path import isfile, join

path = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais"
files = [f for f in listdir(path) if isfile(join(path, f))]

for f in files:
  path_to_zip_file = f"/content/drive/MyDrive/geotesouro/data/transferencias/mensais/{f}"
  directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais/data/"

  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

['201401_Transferencias.zip',
 '201402_Transferencias.zip',
 '201403_Transferencias.zip',
 '201404_Transferencias.zip',
 '201406_Transferencias.zip',
 '201407_Transferencias.zip',
 '201405_Transferencias.zip',
 '201409_Transferencias.zip',
 '201408_Transferencias.zip',
 '201410_Transferencias.zip',
 '201411_Transferencias.zip',
 '201501_Transferencias.zip',
 '201412_Transferencias.zip',
 '201502_Transferencias.zip',
 '201504_Transferencias.zip',
 '201503_Transferencias.zip',
 '201506_Transferencias.zip',
 '201505_Transferencias.zip',
 '201508_Transferencias.zip',
 '201507_Transferencias.zip',
 '201509_Transferencias.zip',
 '201510_Transferencias.zip',
 '201511_Transferencias.zip',
 '201512_Transferencias.zip',
 '201601_Transferencias.zip',
 '201602_Transferencias.zip',
 '201603_Transferencias.zip',
 '201604_Transferencias.zip',
 '201605_Transferencias.zip',
 '201606_Transferencias.zip',
 '201607_Transferencias.zip',
 '201608_Transferencias.zip',
 '201610_Transferencias.zip',
 '201609_T

In [ ]:
import pandas as pd 
from tqdm import tqdm

path = "/content/drive/MyDrive/geotesouro/data/transferencias/mensais/data"
files = [f for f in listdir(path) if isfile(join(path, f))]
transferencias_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "linguagem_cidad", "valor_transf"])

for i in tqdm(range(len(files))):

  f = files[i]
  transferencias = pd.read_csv(f"/content/drive/MyDrive/geotesouro/data/transferencias/mensais/data/{f}", encoding='latin-1', sep = ";", error_bad_lines=False)
  transferencias = transferencias[transferencias["CÓDIGO MUNICÍPIO SIAFI"].notnull()]
  transferencias = transferencias[["ANO / MÊS", "CÓDIGO MUNICÍPIO SIAFI", 'LINGUAGEM CIDADÃ', "VALOR TRANSFERIDO"]].rename(columns = {"ANO / MÊS": "ano_mes", "CÓDIGO MUNICÍPIO SIAFI": "siafi_id", 
                                                                                                                   'LINGUAGEM CIDADÃ': "linguagem_cidad", "VALOR TRANSFERIDO": "valor_transf"})

  transferencias["siafi_id"] = transferencias["siafi_id"].astype(int)
  transferencias["valor_transf"] = transferencias["valor_transf"].replace(",", ".", regex=True).astype(float)
  transferencias = transferencias.groupby(['ano_mes','siafi_id', 'linguagem_cidad']).sum().reset_index()
  transferencias_agg = pd.concat([transferencias_agg, transferencias], ignore_index=True)

  0%|          | 0/96 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,12,14,19,23,25,27,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
  1%|          | 1/96 [00:01<01:41,  1.07s/it]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,12,14,19,21,23,25,27,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 47%|████▋     | 45/96 [01:32<01:50,  2.18s/it]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (12,14,19,21,23

In [ ]:
transferencias_agg

,ano_mes,siafi_id,linguagem_cidad,valor_transf
0,201401,1,CIDE - Combustíveis,11015.84
1,201401,1,FARMÁCIA BÁSICA,17821.53
2,201401,1,FPM - CF art. 159,1356709.00
3,201401,1,FUNDEB,608506.83
4,201401,1,PAB Fixo,98471.33
...,...,...,...,...
7158352,202112,9997,TETO MAC,1250.00
7158353,202112,9997,Transferência - ITR - Municípios,92395.18
7158354,202112,9997,Transferências - LC n.º 87/96 e 115/2003,7225.59
7158355,202112,9997,Vigilância Sanitária,1000.00


In [ ]:
transferencias_agg.to_sql('transferencias', con = conn)

## Emendas Parlamentares 

In [ ]:
emendas = pd.read_csv("/content/drive/MyDrive/geotesouro/data/emendas_parlamentares/Emendas.csv", encoding='latin-1', sep = ";", error_bad_lines=False)

In [ ]:
emendas.to_sql('emendas', con = conn, if_exists='replace', index=False)

## Benefícios ao Cidadão

In [ ]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Garantia-Safra

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2021-12-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/garantia-safra/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips'
  wget.download(url, out = path)


100%|██████████| 1/1 [00:08<00:00,  8.70s/it]


In [ ]:
import zipfile
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips/'
extension = 'zip'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

for f in tqdm(result):
  path_to_zip_file = f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips/{f}"
  directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips/files/"

  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

100%|██████████| 105/105 [00:48<00:00,  2.17it/s]


In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips/files/'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

df_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas", "valor_parcela_agg"])

for f in tqdm(result):
  df = pd.read_csv(f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/garantia_safra/zips/files/{f}", encoding='latin-1', sep = ";", error_bad_lines=False)
  df = df[["MÊS REFERÊNCIA", "CÓDIGO MUNICÍPIO SIAFI", "VALOR PARCELA"]]
  df["VALOR PARCELA"] = df["VALOR PARCELA"].replace(",", ".", regex = True)
  df = df.rename(columns = {"MÊS REFERÊNCIA": "ano_mes", "CÓDIGO MUNICÍPIO SIAFI": "siafi_id", "VALOR PARCELA": "valor_parcela_agg"})
  df["valor_parcela_agg"] = df["valor_parcela_agg"].astype(float)
  df = df.groupby(["ano_mes", "siafi_id"]).agg({"siafi_id": 'count', "valor_parcela_agg": 'sum'}).rename(columns = {"siafi_id": "cont_parcelas"}).reset_index()

  df_agg = pd.concat([df_agg, df], ignore_index = True)

  0%|          | 0/105 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 105/105 [01:45<00:00,  1.01s/it]


In [ ]:
df_agg.to_sql('benef_cidadad - garantia_safra', con = conn, if_exists='replace', index=False)

### Seguro Defeso

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2016-01-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/seguro-defeso/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro-defeso'
  wget.download(url, out = path)

In [ ]:
import zipfile
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro_defeso/zips/'
extension = 'zip'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

for f in tqdm(result):
  path_to_zip_file = f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro_defeso/zips/{f}"
  directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro_defeso/zips/files/"

  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

100%|██████████| 108/108 [03:45<00:00,  2.09s/it]


In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro_defeso/zips/files/'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

df_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas", "valor_parcela_agg"])

for f in tqdm(result):
  df = pd.read_csv(f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/seguro_defeso/zips/files/{f}", encoding='latin-1', sep = ";", error_bad_lines=False)
  df = df[["MÊS REFERÊNCIA", "CÓDIGO MUNICÍPIO SIAFI", "VALOR PARCELA"]]
  df["VALOR PARCELA"] = df["VALOR PARCELA"].replace(",", ".", regex = True)
  df = df.rename(columns = {"MÊS REFERÊNCIA": "ano_mes", "CÓDIGO MUNICÍPIO SIAFI": "siafi_id", "VALOR PARCELA": "valor_parcela_agg"})
  df["valor_parcela_agg"] = df["valor_parcela_agg"].astype(float)
  df = df.groupby(["ano_mes", "siafi_id"]).agg({"siafi_id": 'count', "valor_parcela_agg": 'sum'}).rename(columns = {"siafi_id": "cont_parcelas"}).reset_index()

  df_agg = pd.concat([df_agg, df], ignore_index = True)

  0%|          | 0/108 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 108/108 [01:40<00:00,  1.07it/s]


In [ ]:
df_agg.to_sql('benef_cidadad - seguro_defeso', con = conn, if_exists='replace', index=False)

### BPC

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2019-01-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/bpc/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips'
  wget.download(url, out = path)

In [ ]:
import zipfile
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips'
extension = 'zip'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

for f in tqdm(result):
  path_to_zip_file = f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips/{f}"
  directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips/files/"

  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

100%|██████████| 36/36 [08:21<00:00, 13.94s/it]


In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips/files/'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

df_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas", "valor_parcela_agg"])

for f in tqdm(result):
  df = pd.read_csv(f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bpc/zips/files/{f}", encoding='latin-1', sep = ";", error_bad_lines=False)
  df = df[["MÊS COMPETÊNCIA", "CÓDIGO MUNICÍPIO SIAFI", "VALOR PARCELA"]]
  df["VALOR PARCELA"] = df["VALOR PARCELA"].replace(",", ".", regex = True)
  df = df.rename(columns = {"MÊS COMPETÊNCIA": "ano_mes", "CÓDIGO MUNICÍPIO SIAFI": "siafi_id", "VALOR PARCELA": "valor_parcela_agg"})
  df["valor_parcela_agg"] = df["valor_parcela_agg"].astype(float)
  df = df.groupby(["ano_mes", "siafi_id"]).agg({"siafi_id": 'count', "valor_parcela_agg": 'sum'}).rename(columns = {"siafi_id": "cont_parcelas"}).reset_index()

  df_agg = pd.concat([df_agg, df], ignore_index = True)

  0%|          | 0/35 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 35/35 [25:39<00:00, 43.99s/it]


In [ ]:
df_agg.to_sql('benef_cidadad - bpc', con = conn, if_exists='replace', index=False)

### Erradicação do Trabalho Infantil (PETI)

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2013-01-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/peti/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips'
  wget.download(url, out = path)

In [ ]:
import zipfile
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips (1)/'
extension = 'zip'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

for f in tqdm(result):
  path_to_zip_file = f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips (1)/{f}"
  directory_to_extract_to = "/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips (1)/files/"

  with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
      zip_ref.extractall(directory_to_extract_to)

100%|██████████| 103/103 [01:44<00:00,  1.02s/it]


In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm

path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips (1)/files/'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

df_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas", "valor_parcela_agg"])

for f in tqdm(result):
  df = pd.read_csv(f"/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/peti/zips (1)/files/{f}", encoding='latin-1', sep = ";", error_bad_lines=False)
  df = df[["MÊS REFERÊNCIA", "CÓDIGO SIAFI MUNICÍPIO", "VALOR PARCELA"]]
  df["VALOR PARCELA"] = df["VALOR PARCELA"].replace(",", ".", regex = True)
  df = df.rename(columns = {"MÊS REFERÊNCIA": "ano_mes", "CÓDIGO SIAFI MUNICÍPIO": "siafi_id", "VALOR PARCELA": "valor_parcela_agg"})
  df["valor_parcela_agg"] = df["valor_parcela_agg"].astype(float)
  df = df.groupby(["ano_mes", "siafi_id"]).agg({"siafi_id": 'count', "valor_parcela_agg": 'sum'}).rename(columns = {"siafi_id": "cont_parcelas"}).reset_index()

  df_agg = pd.concat([df_agg, df], ignore_index = True)

 88%|████████▊ | 91/103 [00:04<00:00, 43.95it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
100%|██████████| 103/103 [00:04<00:00, 23.06it/s]


In [ ]:
df_agg.to_sql('benef_cidadad - peti', con = conn, if_exists='replace', index=False)

### Auxílio Emergencial 

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2013-01-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bolsa_familia/zips'
  wget.download(url, out = path)

https://www.portaltransparencia.gov.br/download-de-dados/auxilio-emergencial/202004

### Bolsa Família

In [ ]:
import wget
import pandas as pd 
from tqdm import tqdm

list_dates = pd.date_range(start='2013-01-01', end='2021-12-31', freq = "M").strftime("%Y%m").to_list()

for d in tqdm(range(len(list_dates))):

  url = f'https://www.portaltransparencia.gov.br/download-de-dados/bolsa-familia-pagamentos/{list_dates[d]}'
  path = '/content/drive/MyDrive/geotesouro/data/beneficios_cidadaos/bolsa_familia/zips'
  wget.download(url, out = path)